In [6]:
import requests
import time
import json
from collections import deque

In [7]:
BASE_URL = "http://35.200.185.69:8000/v3/autocomplete?query="

In [8]:
RATE_LIMIT_SLEEP = 2  # Adjust based on API behavior

In [9]:
# Output file to store results
OUTPUT_FILE = "v3_names.json"

# Store discovered names
found_names = set()


In [10]:
request_count = 0

def fetch_names(query):
    global request_count  # Ensure we're modifying the global variable
    request_count += 1  # Increment request count

    try:
        response = requests.get(BASE_URL + query)
        print(f"Request #{request_count}: {BASE_URL}{query}")  # Debug print
        if response.status_code == 200:
            return response.json().get("results", [])
        elif response.status_code == 429:
            print("Rate limit hit. Sleeping...")
            time.sleep(RATE_LIMIT_SLEEP)
            return fetch_names(query)  # Recursive call still increments count
        else:
            print(f"Error {response.status_code}: {response.text}")
            return []
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []

def extract_names():
    """Extract all names using BFS traversal."""
    queue = deque([chr(i) for i in range(97, 123)])  # Start with a-z

    while queue:
        query = queue.popleft()
        print(f"Processing query: {query}, Queue size: {len(queue)}")  # Debug log
        suggestions = fetch_names(query)

        for name in suggestions:
            if name not in found_names:
                found_names.add(name)
                queue.append(name)  # Explore further completions

        # Save progress periodically
        with open(OUTPUT_FILE, "w") as f:
            json.dump(list(found_names), f, indent=4)

        time.sleep(0.5)  # Increased delay to prevent overwhelming the server

if __name__ == "__main__":
    extract_names()
    print(f"Extraction completed. {len(found_names)} names saved to {OUTPUT_FILE}.")

Processing query: a, Queue size: 25
Request #1: http://35.200.185.69:8000/v3/autocomplete?query=a
Processing query: b, Queue size: 39
Request #2: http://35.200.185.69:8000/v3/autocomplete?query=b
Processing query: c, Queue size: 53
Request #3: http://35.200.185.69:8000/v3/autocomplete?query=c
Processing query: d, Queue size: 67
Request #4: http://35.200.185.69:8000/v3/autocomplete?query=d
Processing query: e, Queue size: 81
Request #5: http://35.200.185.69:8000/v3/autocomplete?query=e
Processing query: f, Queue size: 95
Request #6: http://35.200.185.69:8000/v3/autocomplete?query=f
Processing query: g, Queue size: 109
Request #7: http://35.200.185.69:8000/v3/autocomplete?query=g
Processing query: h, Queue size: 123
Request #8: http://35.200.185.69:8000/v3/autocomplete?query=h
Processing query: i, Queue size: 137
Request #9: http://35.200.185.69:8000/v3/autocomplete?query=i
Processing query: j, Queue size: 151
Request #10: http://35.200.185.69:8000/v3/autocomplete?query=j
Processing quer